# Savoir sélectionner des données

Nous allons voir comment lire le dump de Wikidata et y sélectionner des données pour fabriquer notre propre CSV.

## Comment lire le Json du dump de Wikidata ?

On retire les lignes vides ou qui ne contiennent pas d'enregistrement et la virgule à la fin.

In [ ]:
import time
import json

start = time.time()

wikiFile = sc.textFile('wikidata-dump-2017-04-24.json')

def extractProperty(entity, prop):
    try: return entity.get('claims').get(prop)[0].get('mainsnak').get('datavalue').get('value').get('id') 
    except: return ''
    
def extractLabel(entity, lang):
    try: return entity.get('labels').get(lang).get('value')
    except: return ''
    
def loadJsonAndSkipError(string):
    try: return [json.loads(string)]
    except: return []

rddExtractWikidata = wikiFile \
  .filter(lambda x: '{' in x) \
  .flatMap(lambda x: loadJsonAndSkipError(x[:-1])) \
  .map(lambda x: ( int(x.get('id')[1:]),   
                      {
                          'id' : x.get('id'),
                          'label_en': extractLabel(x,'en') ,
                           'P734': extractProperty(x, 'P734')
                      }
                 )\
       )  
    
#print(rddExtractWikidata.take(5))

def changeKey(x,newKey):
    K, V = x
    if V[newKey] == '' :
        return []
    else:        
        return [(int(V[newKey][1:]), V)]

rddP734 =  rddExtractWikidata.flatMap(lambda x: changeKey(x,'P734'))
#print(rddP734.take(5))

rddLabel = rddExtractWikidata.mapValues(lambda v: 
                              {'P734_Label_en': v['label_en']}
                          )
#print(rddLabel.take(5))

#And finally the join
rddJoinP734 = rddP734.join(rddLabel)
#print(rddJoinP734.take(5))

#Organize header of CSV
rddForDataFrame = rddJoinP734.map(lambda x:  
                                       Row(                                           
                                           idWikidata=x[1][0]["id"], 
                                           label_en=x[1][0]["label_en"], 
                                           P734=x[1][0]["P734"], 
                                           P734_Label_en=x[1][1]["P734_Label_en"]
                                          )
                                      )
#Create the dataframe
df = spark.createDataFrame(rddForDataFrame)

#Save the dataframe 
#in one file
#df.coalesce(1).write.csv('rddForDataFrameCSV',header=True)
df.write.csv('folderFinalResultCSV',header=True)

end = time.time()
print(end - start, 'seconds')